In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

import seaborn as sns
sns.set_context('notebook')
sns.set(style="whitegrid", font_scale=1.5)
sns.despine()
sns.set_color_codes()

import matplotlib.pyplot as plt
plt.xlim(0, 1)
plt.ylim(0, None)
from matplotlib.ticker import MaxNLocator
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = [20, 5]

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import logging
logging.basicConfig(level=logging.INFO, stream=sys.stdout)

import json
import vtk
import numpy as np
import nibabel as nib
from six import iteritems
from scipy.stats import norm
from nibabel.affines import apply_affine
from vtk.util import numpy_support as ns
from dipy.tracking.streamline import values_from_volume
from dipy.tracking.metrics import winding

In [2]:
main_folder = '/Volumes/LaCieThree/MAR/'
dirs = os.listdir(main_folder)

In [3]:
def load_nii(fname):
    img = nib.load(fname)
    return img.get_fdata(), img.affine, img.header.get_zooms()

def read_vtk(filename):
    if filename.endswith('xml') or filename.endswith('vtp'):
        polydata_reader = vtk.vtkXMLPolyDataReader()
    else:
        polydata_reader = vtk.vtkPolyDataReader()

    polydata_reader.SetFileName(filename)
    polydata_reader.Update()

    polydata = polydata_reader.GetOutput()

    return vtkpolydata_to_tracts(polydata)

def vtkpolydata_to_tracts(polydata):
    result = {'lines': ns.vtk_to_numpy(polydata.GetLines().GetData()),
              'points': ns.vtk_to_numpy(polydata.GetPoints().GetData()),
              'numberOfLines': polydata.GetNumberOfLines()
             }

    data = {}
    if polydata.GetPointData().GetScalars():
        data['ActiveScalars'] = polydata.GetPointData().GetScalars().GetName()
    if polydata.GetPointData().GetVectors():
        data['ActiveVectors'] = polydata.GetPointData().GetVectors().GetName()
    if polydata.GetPointData().GetTensors():
        data['ActiveTensors'] = polydata.GetPointData().GetTensors().GetName()

    for i in range(polydata.GetPointData().GetNumberOfArrays()):
        array = polydata.GetPointData().GetArray(i)
        np_array = ns.vtk_to_numpy(array)
        if np_array.ndim == 1:
            np_array = np_array.reshape(len(np_array), 1)
        data[polydata.GetPointData().GetArrayName(i)] = np_array

    result['pointData'] = data

    tracts, data = vtkpolydata_dictionary_to_tracts_and_data(result)
    return tracts, data

def vtkpolydata_dictionary_to_tracts_and_data(dictionary):
    dictionary_keys = {'lines', 'points', 'numberOfLines'}
    if not dictionary_keys.issubset(dictionary):
        raise ValueError("Dictionary must have the keys lines and points" + repr(dictionary))

    tract_data = {}
    tracts = []

    lines = np.asarray(dictionary['lines']).squeeze()
    points = dictionary['points']

    actual_line_index = 0
    number_of_tracts = dictionary['numberOfLines']
    original_lines = []
    for _ in range(number_of_tracts):
        tracts.append(points[lines[actual_line_index + 1:actual_line_index + lines[actual_line_index] + 1]])
        original_lines.append(
            np.array(lines[actual_line_index + 1:actual_line_index + lines[actual_line_index] + 1], copy=True))
        actual_line_index += lines[actual_line_index] + 1

    if 'pointData' in dictionary:
        point_data_keys = [it[0] for it in iteritems(dictionary['pointData']) if isinstance(it[1], np.ndarray)]

        for k in point_data_keys:
            array_data = dictionary['pointData'][k]
            if k not in tract_data:
                tract_data[k] = [array_data[f] for f in original_lines]
            else:
                np.vstack(tract_data[k])
                tract_data[k].extend([array_data[f] for f in original_lines[-number_of_tracts:]])

    return tracts, tract_data

In [4]:
def get_volume(streamlines, shapes, affine, dims):
    if streamlines:
        new_vol = np.zeros(shapes)
        inverse = np.linalg.inv(affine)
        flat = []
        for s in streamlines:
            flat += apply_affine(inverse, np.array(s)).astype(np.int16).tolist()
        return(np.prod(dims) * len(np.unique(np.asarray(flat), axis = 0)))
    

In [5]:
volumes = {}
for current_dir in dirs:
    if os.path.isdir(os.path.join(main_folder,current_dir)):
        volumes[current_dir] = {}
        l5 = None
        s1 = None
        s2 = None
        s3 = None
        s4 = None
        for file in os.listdir(os.path.join(main_folder,current_dir)):
            current_path = os.path.abspath(os.path.join(main_folder, current_dir, file))
            if not file.startswith('.') and '.vtk' in file.lower() and 'cut' in file.lower():
                if 'l5' in current_path.lower():
                    l5 = read_vtk(current_path)[0]
                elif 's1' in current_path.lower():
                    s1 = read_vtk(current_path)[0]
                elif 's2' in current_path.lower():
                    s2 = read_vtk(current_path)[0]
                elif 's3' in current_path.lower():
                    s3 = read_vtk(current_path)[0]
                elif  's4' in current_path.lower():
                    s4 = read_vtk(current_path)[0]
            if not file.startswith('.'):
                if '.nii' in current_path.lower() and 'bzero' in current_path.lower():
                    coro, affine, dims = load_nii(current_path)
        try:
            volumes[current_dir]['L5'] = get_volume(l5, coro.shape, affine, dims)
            volumes[current_dir]['S1'] = get_volume(s1, coro.shape, affine, dims)
            volumes[current_dir]['S2'] = get_volume(s2, coro.shape, affine, dims)
            volumes[current_dir]['S3'] = get_volume(s3, coro.shape, affine, dims)
            volumes[current_dir]['S4'] = get_volume(s4, coro.shape, affine, dims)
        except:
            pass

with open(os.path.join(main_folder, 'volumes_cut.json'), 'w') as fp:
    print(os.path.join(main_folder, 'volumes_cut.json'))
    json.dump(volumes, fp)

/Volumes/LaCieThree/MAR/volumes_cut.json
